In [2]:
!pip -q install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.6 MB/s eta 0:00:00


In [11]:
import pandas as pd
pd.set_option('display.max_columns', 0)
import numpy as np
from datetime import datetime, timedelta
import math, random
from math import radians, sin, cos, sqrt, atan2
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_absolute_error, mean_squared_error

SEED = 42
rng = np.random.default_rng(SEED)
random.seed(SEED)
np.random.seed(SEED)

In [4]:
def random_coord(base_lat, base_lon, radius_km=5, n=1):
    lats, lons = [], []
    for _ in range(n):
        r = radius_km * math.sqrt(random.random())  # uniform inside circle
        theta = random.random() * 2 * math.pi
        dlat = (r * math.cos(theta)) / 111.0
        dlon = (r * math.sin(theta)) / (111.0 * math.cos(math.radians(base_lat)))
        lats.append(base_lat + dlat); lons.append(base_lon + dlon)
    return np.array(lats), np.array(lons)

In [5]:
def seasonality_factor(dt):
    m = dt.month
    return 1.0 + {1:-0.01,2:-0.005,3:0.01,4:0.02,5:0.03,6:0.035,7:0.03,8:0.02,9:0.0,10:-0.005,11:-0.01,12:-0.015}[m]

In [6]:
cbsa_list = [
    {"cbsa":"31080","cbsa_name":"Los Angeles-Long Beach-Anaheim, CA","base_lat":34.05,"base_lon":-118.25},
    {"cbsa":"35620","cbsa_name":"New York-Newark-Jersey City, NY-NJ-PA","base_lat":40.71,"base_lon":-74.01},
]
counties = []
for cb in cbsa_list:
    for i in range(3):
        counties.append({
            "cbsa": cb["cbsa"],
            "cbsa_name": cb["cbsa_name"],
            "county_fips": f"{int(cb['cbsa'])%1000:03d}{i+1:02d}",
            "county_name": f"County_{i+1}_{cb['cbsa'][-2:]}",
            "base_lat": cb["base_lat"] + rng.normal(0, 0.05),
            "base_lon": cb["base_lon"] + rng.normal(0, 0.05)
        })

In [7]:
zips = []
for c in counties:
    for j in range(3):
        zips.append({
            "cbsa": c["cbsa"],
            "cbsa_name": c["cbsa_name"],
            "county_fips": c["county_fips"],
            "county_name": c["county_name"],
            "zip": f"{rng.integers(10000, 99999)}",
            "neighborhood_id": f"NB_{c['county_fips']}_{j+1}",
            "base_lat": c["base_lat"] + rng.normal(0, 0.03),
            "base_lon": c["base_lon"] + rng.normal(0, 0.03)
        })

In [8]:
n_props = 60  # a few dozen rows
rows = []
start_date = pd.Timestamp("2022-01-01")
end_date   = pd.Timestamp("2024-12-31")
sale_dates = rng.choice(pd.date_range(start_date, end_date, freq="7D"), size=n_props, replace=False)
sale_dates.sort()

In [9]:
for i in range(n_props):
    z = zips[rng.integers(0, len(zips))]
    property_type = rng.choice(["SFH","Townhouse","Condo"], p=[0.6,0.2,0.2])
    bedrooms  = int(rng.integers(1,6))
    bathrooms = np.round(rng.uniform(1, 4), 1)
    sqft = int(max(450, rng.normal(1700 if property_type!="Condo" else 1100, 350)))
    lot_size = int(max(800, rng.normal(6000 if property_type!="Condo" else 0, 1500)))
    year_built = int(rng.integers(1950, 2024))
    condition_score = int(np.clip(rng.normal(70, 15), 30, 100))
    lat, lon = random_coord(z["base_lat"], z["base_lon"], radius_km=5, n=1); lat=float(lat[0]); lon=float(lon[0])

    # market context (as-of listing/sale)
    zip_median_ppsf   = rng.normal(600, 120) if z["cbsa"]=="35620" else rng.normal(450, 90)
    county_median_ppsf= zip_median_ppsf * rng.normal(0.98, 0.03)
    cbsa_median_ppsf  = zip_median_ppsf * rng.normal(1.02, 0.05)
    zip_yoy = rng.normal(0.02, 0.03)
    county_yoy = zip_yoy + rng.normal(0.0, 0.01)
    cbsa_yoy   = zip_yoy + rng.normal(0.0, 0.015)
    inventory_zip = max(10, int(rng.normal(120, 30)))
    dom_zip      = max(10, int(rng.normal(35, 8)))

    # prior sale
    had_prior = rng.random() < 0.65
    months_since_prior_sale = int(rng.integers(12, 180)) if had_prior else np.nan
    prior_sale_price = np.nan
    if had_prior:
        years_back = months_since_prior_sale / 12.0
        back_factor = (1 + zip_yoy) ** years_back
        base_ppsf_now   = zip_median_ppsf * rng.normal(1.0, 0.1)
        base_ppsf_prior = base_ppsf_now / back_factor
        prior_sale_price = max(50_000, int(base_ppsf_prior * sqft + rng.normal(0, 20_000)))

    # target (sale price)
    sale_date = pd.Timestamp(sale_dates[i])
    seasonal  = seasonality_factor(sale_date)
    ppsf_now  = zip_median_ppsf * rng.normal(1.0 + 0.0005*(condition_score-70), 0.07) * seasonal
    if property_type == "Condo": ppsf_now *= 0.9
    if bedrooms >= 4 and sqft > 2200: ppsf_now *= 1.05
    price = int(np.clip(ppsf_now * sqft + rng.normal(0, 25_000), 80_000, 2_500_000))

    price_per_sqft = price / max(500, sqft)
    relative_to_zip_median = price_per_sqft / zip_median_ppsf
    month = sale_date.month
    quarter = (month-1)//3 + 1

    rows.append({
        "property_id": f"PID_{i:04d}",
        "sale_date": sale_date,
        "sale_price": price,
        # physical
        "property_type": property_type,
        "bedrooms": bedrooms, "bathrooms": bathrooms, "sqft": sqft, "lot_size": lot_size,
        "year_built": year_built, "condition_score": condition_score,
        # location/codes
        "lat": lat, "lon": lon,
        "zip": z["zip"], "neighborhood_id": z["neighborhood_id"],
        "county_fips": z["county_fips"], "county_name": z["county_name"],
        "cbsa": z["cbsa"], "cbsa_name": z["cbsa_name"],
        # market context (as-of)
        "zip_median_ppsf_90d": round(zip_median_ppsf, 2),
        "county_median_ppsf_90d": round(county_median_ppsf, 2),
        "cbsa_median_ppsf_90d": round(cbsa_median_ppsf, 2),
        "zip_yoy_appreciation": round(zip_yoy, 4),
        "county_yoy_appreciation": round(county_yoy, 4),
        "cbsa_yoy_appreciation": round(cbsa_yoy, 4),
        "zip_inventory_active": inventory_zip,
        "zip_days_on_market": dom_zip,
        # prior sale
        "prior_sale_price": prior_sale_price,
        "months_since_prior_sale": months_since_prior_sale,
        # engineered
        "price_per_sqft": round(price_per_sqft, 2),
        "relative_to_zip_median": round(relative_to_zip_median, 3),
        "sale_year": sale_date.year, "sale_month": month, "sale_quarter": quarter,
        "season_spring_summer": 1 if month in [3,4,5,6,7,8] else 0,
    })

df = pd.DataFrame(rows).sort_values("sale_date").reset_index(drop=True)
print(df.shape)
df.head()

(60, 34)


,property_id,sale_date,sale_price,property_type,bedrooms,bathrooms,sqft,lot_size,year_built,condition_score,...,zip_inventory_active,zip_days_on_market,prior_sale_price,months_since_prior_sale,price_per_sqft,relative_to_zip_median,sale_year,sale_month,sale_quarter,season_spring_summer
0,PID_0000,2022-01-22,714777,Townhouse,1,2.9,1341,6268,1993,73,...,92,31,387221.0,114.0,533.02,0.931,2022,1,1,0
1,PID_0001,2022-01-29,1282280,SFH,2,2.5,1845,4515,1986,38,...,124,33,1098691.0,65.0,695.00,1.099,2022,1,1,0
2,PID_0002,2022-02-19,562932,SFH,4,1.7,1362,4912,2017,100,...,118,29,277466.0,101.0,413.31,1.099,2022,2,1,0
3,PID_0003,2022-02-26,731666,Condo,2,2.6,1557,800,2016,63,...,169,48,713580.0,137.0,469.92,0.855,2022,2,1,0
4,PID_0004,2022-03-12,551770,Condo,3,1.4,1218,2111,1987,71,...,109,42,NaN,NaN,453.01,0.892,2022,3,1,1


In [12]:
df

,property_id,sale_date,sale_price,property_type,bedrooms,bathrooms,sqft,lot_size,year_built,condition_score,lat,lon,zip,neighborhood_id,county_fips,county_name,cbsa,cbsa_name,zip_median_ppsf_90d,county_median_ppsf_90d,cbsa_median_ppsf_90d,zip_yoy_appreciation,county_yoy_appreciation,cbsa_yoy_appreciation,zip_inventory_active,zip_days_on_market,prior_sale_price,months_since_prior_sale,price_per_sqft,relative_to_zip_median,sale_year,sale_month,sale_quarter,season_spring_summer
0,PID_0000,2022-01-22,714777,Townhouse,1,2.9,1341,6268,1993,73,34.052274,-118.320500,13942,NB_08003_2,08003,County_3_80,31080,"Los Angeles-Long Beach-Anaheim, CA",572.33,575.22,593.99,0.0639,0.0520,0.0543,92,31,387221.0,114.0,533.02,0.931,2022,1,1,0
1,PID_0001,2022-01-29,1282280,SFH,2,2.5,1845,4515,1986,38,40.695615,-73.975579,71782,NB_62001_2,62001,County_1_20,35620,"New York-Newark-Jersey City, NY-NJ-PA",632.13,604.07,631.64,0.0016,0.0002,0.0176,124,33,1098691.0,65.0,695.00,1.099,2022,1,1,0
2,PID_0002,2022-02-19,562932,SFH,4,1.7,1362,4912,2017,100,34.022276,-118.332726,67947,NB_08001_2,08001,County_1_80,31080,"Los Angeles-Long Beach-Anaheim, CA",376.08,378.01,366.62,0.0479,0.0518,0.0456,118,29,277466.0,101.0,413.31,1.099,2022,2,1,0
3,PID_0003,2022-02-26,731666,Condo,2,2.6,1557,800,2016,63,34.135409,-118.261282,80340,NB_08001_1,08001,County_1_80,31080,"Los Angeles-Long Beach-Anaheim, CA",549.57,545.64,602.76,0.0255,0.0133,0.0050,169,48,713580.0,137.0,469.92,0.855,2022,2,1,0
4,PID_0004,2022-03-12,551770,Condo,3,1.4,1218,2111,1987,71,34.110725,-118.216824,15743,NB_08002_1,08002,County_2_80,31080,"Los Angeles-Long Beach-Anaheim, CA",507.95,466.55,516.88,-0.0053,-0.0175,-0.0185,109,42,NaN,NaN,453.01,0.892,2022,3,1,1
5,PID_0005,2022-03-19,1218888,SFH,2,3.0,2168,5768,2021,66,40.713753,-73.980300,73464,NB_62003_3,62003,County_3_20,35620,"New York-Newark-Jersey City, NY-NJ-PA",629.10,619.85,607.57,0.0227,0.0250,0.0605,176,28,NaN,NaN,562.22,0.894,2022,3,1,1
6,PID_0006,2022-04-02,871688,SFH,4,3.5,1471,2779,1958,67,40.746698,-73.976033,52440,NB_62003_2,62003,County_3_20,35620,"New York-Newark-Jersey City, NY-NJ-PA",472.51,455.56,461.24,0.0172,-0.0004,-0.0048,183,24,NaN,NaN,592.58,1.254,2022,4,2,1
7,PID_0007,2022-04-16,1580838,SFH,1,1.4,2305,4519,2018,81,40.727768,-73.965531,22577,NB_62003_1,62003,County_3_20,35620,"New York-Newark-Jersey City, NY-NJ-PA",652.17,631.77,660.85,-0.0212,-0.0236,-0.0252,126,30,1768772.0,36.0,685.83,1.052,2022,4,2,1
8,PID_0008,2022-04-30,488508,Condo,5,1.1,1065,3139,1997,75,40.697812,-74.041352,84902,NB_62002_2,62002,County_2_20,35620,"New York-Newark-Jersey City, NY-NJ-PA",508.43,481.30,548.88,0.0279,0.0327,0.0017,147,38,NaN,NaN,458.69,0.902,2022,4,2,1
9,PID_0009,2022-06-18,447987,Townhouse,4,1.4,1611,6228,2007,92,40.732150,-74.004112,71782,NB_62001_2,62001,County_1_20,35620,"New York-Newark-Jersey City, NY-NJ-PA",292.00,284.09,300.42,0.0289,0.0252,0.0025,129,48,361062.0,96.0,278.08,0.952,2022,6,2,1


In [13]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    dlat = radians(lat2-lat1)
    dlon = radians(lon2-lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1))*cos(radians(lat2))*sin(dlon/2)**2
    return 2 * R * atan2(sqrt(a), sqrt(1-a))

In [14]:
df = df.sort_values("sale_date").reset_index(drop=True)

In [15]:
medians = []
for i, r in df.iterrows():
    cutoff = r["sale_date"] - pd.Timedelta(days=180)
    pool = df[(df["sale_date"] < r["sale_date"]) & (df["sale_date"] >= cutoff)]
    if len(pool):
        d = pool.apply(lambda x: haversine(r["lat"], r["lon"], x["lat"], x["lon"]), axis=1)
        pool2 = pool[d < 5.0]
        medians.append(pool2["sale_price"].median() if len(pool2) else np.nan)
    else:
        medians.append(np.nan)

df["comp_median_price_5km_180d"] = pd.Series(medians).fillna(df["sale_price"].median())
df.head()

,property_id,sale_date,sale_price,property_type,bedrooms,bathrooms,sqft,lot_size,year_built,condition_score,lat,lon,zip,neighborhood_id,county_fips,county_name,cbsa,cbsa_name,zip_median_ppsf_90d,county_median_ppsf_90d,cbsa_median_ppsf_90d,zip_yoy_appreciation,county_yoy_appreciation,cbsa_yoy_appreciation,zip_inventory_active,zip_days_on_market,prior_sale_price,months_since_prior_sale,price_per_sqft,relative_to_zip_median,sale_year,sale_month,sale_quarter,season_spring_summer,comp_median_price_5km_180d
0,PID_0000,2022-01-22,714777,Townhouse,1,2.9,1341,6268,1993,73,34.052274,-118.320500,13942,NB_08003_2,08003,County_3_80,31080,"Los Angeles-Long Beach-Anaheim, CA",572.33,575.22,593.99,0.0639,0.0520,0.0543,92,31,387221.0,114.0,533.02,0.931,2022,1,1,0,769952.5
1,PID_0001,2022-01-29,1282280,SFH,2,2.5,1845,4515,1986,38,40.695615,-73.975579,71782,NB_62001_2,62001,County_1_20,35620,"New York-Newark-Jersey City, NY-NJ-PA",632.13,604.07,631.64,0.0016,0.0002,0.0176,124,33,1098691.0,65.0,695.00,1.099,2022,1,1,0,769952.5
2,PID_0002,2022-02-19,562932,SFH,4,1.7,1362,4912,2017,100,34.022276,-118.332726,67947,NB_08001_2,08001,County_1_80,31080,"Los Angeles-Long Beach-Anaheim, CA",376.08,378.01,366.62,0.0479,0.0518,0.0456,118,29,277466.0,101.0,413.31,1.099,2022,2,1,0,714777.0
3,PID_0003,2022-02-26,731666,Condo,2,2.6,1557,800,2016,63,34.135409,-118.261282,80340,NB_08001_1,08001,County_1_80,31080,"Los Angeles-Long Beach-Anaheim, CA",549.57,545.64,602.76,0.0255,0.0133,0.0050,169,48,713580.0,137.0,469.92,0.855,2022,2,1,0,769952.5
4,PID_0004,2022-03-12,551770,Condo,3,1.4,1218,2111,1987,71,34.110725,-118.216824,15743,NB_08002_1,08002,County_2_80,31080,"Los Angeles-Long Beach-Anaheim, CA",507.95,466.55,516.88,-0.0053,-0.0175,-0.0185,109,42,NaN,NaN,453.01,0.892,2022,3,1,1,731666.0


In [16]:
train_end = pd.Timestamp("2023-06-30")
val_end   = pd.Timestamp("2023-12-31")

train_df = df[df["sale_date"] <= train_end].copy()
val_df   = df[(df["sale_date"] > train_end) & (df["sale_date"] <= val_end)].copy()
test_df  = df[df["sale_date"] > val_end].copy()

for name, part in [("train",train_df),("val",val_df),("test",test_df)]:
    print(name, len(part),
          part["sale_date"].min().date() if len(part) else None,
          part["sale_date"].max().date() if len(part) else None)

train 34 2022-01-22 2023-06-17
val 5 2023-07-22 2023-10-14
test 21 2024-01-06 2024-12-28


In [17]:
train_df["log_price"] = np.log(train_df["sale_price"])
val_df["log_price"]   = np.log(val_df["sale_price"])
test_df["log_price"]  = np.log(test_df["sale_price"])

# Feature columns (mix of numeric + categorical)
cat_cols = ["property_type","zip","neighborhood_id","county_fips","county_name","cbsa","cbsa_name"]
num_cols = [
    "bedrooms","bathrooms","sqft","lot_size","year_built","condition_score",
    "lat","lon",
    "zip_median_ppsf_90d","county_median_ppsf_90d","cbsa_median_ppsf_90d",
    "zip_yoy_appreciation","county_yoy_appreciation","cbsa_yoy_appreciation",
    "zip_inventory_active","zip_days_on_market",
    "prior_sale_price","months_since_prior_sale",
    "price_per_sqft","relative_to_zip_median",
    "sale_year","sale_month","sale_quarter","season_spring_summer",
    "comp_median_price_5km_180d"
]
feature_cols = cat_cols + num_cols

In [19]:
# Build Pools (CatBoost native categorical handling)

# Create a list of column names for feature_cols
feature_cols = cat_cols + num_cols

# Get the indices of the categorical columns within the feature_cols list
cat_feature_indices = [feature_cols.index(c) for c in cat_cols]

train_pool = Pool(
    train_df[feature_cols],
    label=train_df["log_price"],
    cat_features=cat_feature_indices  # Use the correct indices
)
val_pool = Pool(
    val_df[feature_cols],
    label=val_df["log_price"],
    cat_features=cat_feature_indices # Use the correct indices
)

model = CatBoostRegressor(
    loss_function="RMSE",
    eval_metric="RMSE",
    learning_rate=0.05,
    depth=8,
    l2_leaf_reg=3.0,
    random_seed=SEED,
    iterations=5000,
    early_stopping_rounds=200,
    verbose=False
)
model.fit(train_pool, eval_set=val_pool, use_best_model=True)

print("Best iteration:", model.get_best_iteration())
print("Best validation RMSE (log-price):", model.get_best_score()["validation"]["RMSE"])

Best iteration: 163
Best validation RMSE (log-price): 0.16205357134280016


In [22]:
test_pool = Pool(
    test_df[feature_cols],
    label=test_df["log_price"],
    cat_features=[feature_cols.index(c) for c in cat_cols]
)

log_preds = model.predict(test_pool)
preds = np.exp(log_preds)

mae  = mean_absolute_error(test_df["sale_price"], preds)
rmse = mean_squared_error(test_df["sale_price"], preds)
mape = (np.abs(preds - test_df["sale_price"]) / test_df["sale_price"]).mean()

print(f"Test MAE:  ${mae:,.0f}")
print(f"Test RMSE: ${rmse:,.0f}")
print(f"Test MAPE: {100*mape:.2f}%")

Test MAE:  $191,096
Test RMSE: $54,982,664,476
Test MAPE: 27.39%


In [23]:
fi = pd.DataFrame({
    "feature": feature_cols,
    "importance": model.get_feature_importance(train_pool)
}).sort_values("importance", ascending=False)

print("Top 15 features:")
display(fi.head(15))

print("\nSample predictions (first 10 test rows):")
sample = test_df[["property_id","sale_date","sale_price"] + feature_cols].head(10).copy()
sample["pred_price"] = preds[:len(sample)]
sample["pct_error"]  = 100*(sample["pred_price"] - sample["sale_price"]) / sample["sale_price"]
display(sample[["property_id","sale_date","sale_price","pred_price","pct_error"]])


Top 15 features:


,feature,importance
15,zip_median_ppsf_90d,34.749116
9,sqft,15.706569
17,cbsa_median_ppsf_90d,7.325087
16,county_median_ppsf_90d,5.290371
25,price_per_sqft,5.202846
23,prior_sale_price,3.689386
4,county_name,2.625658
0,property_type,2.502385
8,bathrooms,1.835504
14,lon,1.772392



Sample predictions (first 10 test rows):


,property_id,sale_date,sale_price,pred_price,pct_error
39,PID_0039,2024-01-06,582052,6.722285e+05,15.492854
40,PID_0040,2024-01-13,390998,6.636062e+05,69.721126
41,PID_0041,2024-01-20,1437809,8.975702e+05,-37.573751
42,PID_0042,2024-01-27,916683,8.722689e+05,-4.845090
43,PID_0043,2024-02-03,761663,7.294371e+05,-4.230988
44,PID_0044,2024-02-10,1073323,1.009342e+06,-5.961023
45,PID_0045,2024-02-17,434950,6.561695e+05,50.860904
46,PID_0046,2024-03-16,1364975,9.662499e+05,-29.211163
47,PID_0047,2024-03-30,608315,8.310158e+05,36.609459
48,PID_0048,2024-04-06,1078695,8.258667e+05,-23.438349


In [24]:
q10 = CatBoostRegressor(
    loss_function="Quantile:alpha=0.10",
    depth=8, learning_rate=0.05, iterations=2000, early_stopping_rounds=200,
    random_seed=SEED, verbose=False
)
q90 = CatBoostRegressor(
    loss_function="Quantile:alpha=0.90",
    depth=8, learning_rate=0.05, iterations=2000, early_stopping_rounds=200,
    random_seed=SEED, verbose=False
)

q10.fit(train_pool, eval_set=val_pool, use_best_model=True)
q90.fit(train_pool, eval_set=val_pool, use_best_model=True)

test_pred_q10 = np.exp(q10.predict(test_pool))
test_pred_q90 = np.exp(q90.predict(test_pool))

pi_df = pd.DataFrame({
    "property_id": test_df["property_id"],
    "sale_price": test_df["sale_price"],
    "pred": preds,
    "pred_p10": test_pred_q10,
    "pred_p90": test_pred_q90
})
pi_df["covered"] = (pi_df["sale_price"] >= pi_df["pred_p10"]) & (pi_df["sale_price"] <= pi_df["pred_p90"])
coverage = pi_df["covered"].mean()
print(f"Empirical PI coverage (10–90%): {coverage:.2%}")

display(pi_df.head(10))


Empirical PI coverage (10–90%): 66.67%


,property_id,sale_price,pred,pred_p10,pred_p90,covered
39,PID_0039,582052,6.722285e+05,515579.138132,8.028367e+05,True
40,PID_0040,390998,6.636062e+05,499148.894317,7.125933e+05,False
41,PID_0041,1437809,8.975702e+05,582554.844220,1.343270e+06,False
42,PID_0042,916683,8.722689e+05,592720.981097,9.903635e+05,True
43,PID_0043,761663,7.294371e+05,510874.849234,7.978985e+05,True
44,PID_0044,1073323,1.009342e+06,614497.981672,1.068683e+06,False
45,PID_0045,434950,6.561695e+05,504761.419837,7.326015e+05,False
46,PID_0046,1364975,9.662499e+05,617992.340609,1.210324e+06,False
47,PID_0047,608315,8.310158e+05,554465.416618,9.154390e+05,True
48,PID_0048,1078695,8.258667e+05,541750.762433,1.227185e+06,True
